In [1]:
# Magic functions
%matplotlib
%load_ext autoreload
%autoreload 2

Using matplotlib backend: TkAgg


In [13]:
%%javascript
IPython.OutputArea.prototype._should_scroll = function(lines) {
    return false;
}

<IPython.core.display.Javascript object>

In [2]:
from calculate import calculate
from tramway.core import *
from tramway.helper.tessellation import cell_plot
from tramway.inference.base import Translocations, distributed, Maps
from tramway.tessellation import *
import numpy as np
from numpy.random import rand
import os
import pandas as pd
from pandas import DataFrame
from snr import infer_snr


D:\git\Stochastic_Integrals_Diffusivity\ito-to-tramway\calculate.py:10: UserWarning: 
This call to matplotlib.use() has no effect because the backend has already
been chosen; matplotlib.use() must be called *before* pylab, matplotlib.pyplot,
or matplotlib.backends is imported for the first time.

The backend was *originally* set to 'TkAgg' by the following code:
  File "c:\python36\lib\runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "c:\python36\lib\runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "c:\python36\lib\site-packages\ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "c:\python36\lib\site-packages\traitlets\config\application.py", line 658, in launch_instance
    app.start()
  File "c:\python36\lib\site-packages\ipykernel\kernelapp.py", line 486, in start
    self.io_loop.start()
  File "c:\python36\lib\site-packages\tornado\platform\asyncio.py", line 112, in start
    self.asyncio_loop.run_foreve

In [17]:




# Constants
csv_file = r".\data\test_mesh_sensitivity\01.csv"
results_folder = r".\data\test_mesh_sensitivity\results" + "\\"
dt = 0.04 # seconds
x_min = 0.0 # in um
x_max = 1.0 # in um
n = 100


# Load the trajectories
file_no_ext, _ = os.path.splitext(csv_file)
rwa_file = '{}.rwa'.format(file_no_ext)

data = pd.read_csv(csv_file, sep=';', skiprows=1, names=['x', 'dx', 'y', 'dy'])
data['n'] = np.zeros(data.shape[0])
data['t'] = np.arange(0., dt * data.shape[0], dt)




# Center seeds
seeds = rand(n, 2)
seeds = x_min + seeds * (x_max - x_min)

# Tesselate
tessellation = Voronoi(Scaler())
tessellation.tessellate(pd.DataFrame(seeds, columns=['x', 'y']))

# Bin the data
cells = CellStats(data, tessellation)
cells.cell_index = tessellation.cell_index(data) # This assigns points to cells (bins)

# Create the analysis tree
label = "n=%i" % n
analyses = Analyses(data)
analyses.add(Analyses(cells), label = label)

# # Save rwa
# save_rwa(rwa_file, analyses, force = True)

# Plot
cell_plot(cells, output_file = file_no_ext + '_mesh_n=%i.png' % n)


# Trajectories with pre-defined jumps
class TranslocationsDx(Translocations):
    def _extract_space(self):
        return np.asarray(self.origins[['dx', 'dy']])
    @property
    def space_cols(self):
        return ['x', 'y']
    @space_cols.setter
    def space_cols(self, cs):
        pass

## Perform inference
# Build a distributed object
d = distributed(cells, new_cell=TranslocationsDx) 

# Run inference, add to cells and save
x = d.run(infer_snr, max_iter=10, localization_error = 1e-3) #, diffusivity_prior = 0.001, min_diffusivity = 0.001)
analyses[label].add(Maps(x, mode='snr'), label='snr')
save_rwa(rwa_file, analyses, force = True, verbose = True)

# Calculate the Bayes factors
calculate(csv_file = csv_file, results_folder = results_folder, bl_output_map = True, dt = dt, snr_label = 'snr')

# Get the presence of forces
# dat_file = results_folder + "_" + file_no_ext + label + ".dat"
# forces = pd.read_csv


writing file: .\data\test_mesh_sensitivity\01.rwa
written analysis tree:
	<class 'pandas.core.frame.DataFrame'>
		'n=100' <class 'tramway.tessellation.base.CellStats'>
			'snr' <class 'tramway.inference.base.Maps'>
Using  .\data\test_mesh_sensitivity\01.rwa


Mean jump along x: <dx>=	-0.021 um


Median D:	0.015 um^2/s
Measured range of D:	[0.0088; 0.022] um^2/s


Inferred <|D'|>:	[0.02086991 0.01095199] um/s


Median zeta_ts:	[-0.4335057  -0.00483096]
Median abs(zeta_sps):	[0.01761177 0.00880065]
Expected zeta_sps:	[0.01615805 0.        ]
Median n:	82
Median min_n:	86
max(min_n):	282


Inferred <alpha dt>:	[-0.02039857 -0.00024419] um





c:\python36\lib\site-packages\pandas\core\indexing.py:194: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)
